In [ ]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#!pip install unidecode
#!pip install datasketch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import unidecode
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict
from collections import Counter
import random
from datasketch import MinHash, MinHashLSH

In [ ]:
def measure_metrics(true_labels, predicted_labels):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(true_labels)):
        golden = str(true_labels[i]).split(", ")
        predicted = str(predicted_labels[i]).split(", ")
        if len(golden) == 1 and len(predicted) == 1 and predicted == golden:
            tn += 1
        elif golden == predicted:
            tp += 1
        elif len(predicted) > len(golden):
            fp += 1
        else:
            fn += 1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1score = 2*tp / (2*tp + fp + fn)
    return accuracy, precision, recall, f1score



def measure_metrics2(true_labels, predicted_labels, include_non_dups=False):
    pre = []
    rec = []
    f1 = []

    for i in range(len(true_labels)):
        tp = 0
        fp = 0
        fn = 0
        
        golden = str(true_labels[i]).split(", ")
        predicted = str(predicted_labels[i]).split(", ")
        
        # Check if there is an intersection before attempting to remove elements
        intersection = list(set(golden).intersection(set(predicted)))
        if intersection:
            to_remove = intersection[0]
            golden.remove(to_remove)
            predicted.remove(to_remove)
            
        if golden or predicted:
            for j in golden:
                if j in predicted:
                    tp += 1
                else:
                    fn += 1

            for j in predicted:
                if j not in golden:
                    fp += 1

            if tp + fp == 0:
                precision = 0
            else:
                precision = tp / (tp + fp)

            if tp + fn == 0:
                recall = 0
            else:
                recall = tp / (tp + fn)

            if precision + recall == 0:
                f1score = 0
            else:
                f1score = 2 * (precision * recall) / (precision + recall)
                
            pre.append(precision)
            rec.append(recall)
            f1.append(f1score)
            
        elif include_non_dups == True:
            pre.append(1)
            rec.append(1)
            f1.append(1)

    return np.mean(pre), np.mean(rec), np.mean(f1)


def replace_non_letters(input_string):
    return re.sub(r"[^a-zA-Z\s]", " ", input_string)

def unidecoded(panda_column):
    return panda_column.astype(str).apply(
        lambda x: unidecode.unidecode(
            x.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue")
             .replace('Ä', 'Ae').replace('Ö', 'Oe').replace('Ü', 'Ue')))

true = [103]
predicted = [103]

measure_metrics2(true, predicted, include_non_dups=True)

In [ ]:
def generate_emails(first_name, last_name):
    # Diese Funktion erzeugt eine Liste von E-Mail-Adressen basierend auf Vor- und Nachnamen
    # Diese Liste enthält verschiedene Formate wie Vorname.Nachname, VNach, Vorname usw.
    emails = []
    emails.append(f"{first_name.lower()}.{last_name.lower()}")
    emails.append(f"{first_name.lower()[0]}{last_name.lower()}")
    emails.append(f"{first_name.lower()}")
    emails.append(f"{first_name.lower()[0]}.{last_name.lower()}")
    emails.append("anderes")
    emails.append("info")
    emails.append(f"{last_name.lower()}")
    emails.append(f"{first_name.lower()}{last_name.lower()}")
    emails.append(f"{first_name.lower()}.{last_name.lower()}1990")  # Vor.NachnameJahr
    return emails

def fake_emails(first_name, last_name):
    # Diese Funktion generiert eine E-Mail-Adresse basierend auf den gegebenen Wahrscheinlichkeiten
    distr = {
        "vor.nach": 55/100,
        "vnach": 10/100,
        "vor": 9/100,
        "v.nach": 6/100,
        "anderes": 6/100,
        "info": 5/100,
        "nach": 4/100,
        "vornach": 3/100,
        "vor.nachjahr": 2/100
    }
    
    # Generiere eine Liste von E-Mail-Adressen
    emails = generate_emails(first_name, last_name)
    
    # Wähle zufällig eine E-Mail-Adresse basierend auf den Wahrscheinlichkeiten aus
    random_percentage = random.random()
    total = 0
    i = 0
    for scenario, percentage in distr.items():
        total += percentage
        if random_percentage <= total:

            return emails[i]  # Gib die E-Mail-Adresse zurück
        else:
            i += 1

def plausible_email(first_name, last_name, email):
    email = email.split("@")[0]
    email = re.sub(r'\d+', '', email)
    plausible_formats = [
        f"{first_name.lower()}.{last_name.lower()}",
        f"{first_name.lower()}{last_name.lower()}",
        f"{first_name.lower()[0]}{last_name.lower()}",
        f"{first_name.lower()}",
        f"{first_name.lower()[0]}.{last_name.lower()}",
        f"{last_name.lower()}",
    ]
    
    if first_name and last_name in email:
        return True
    else:
        return email in plausible_formats

# Beispielvor- und nachname
first_name = "Max"
last_name = "Mustermann"

df = pd.read_excel("Testcases.xlsx")

for vor, nach, email in zip(unidecoded(df["Vorname"]).to_list(), unidecoded(df["Nachname"]).to_list(), unidecoded(df["Email"]).to_list()):
    res = plausible_email(vor, nach, email)
    print(res)
    # Generiere eine E-Mail-Adresse basierend auf den Wahrscheinlichkeiten
    #generated_email = fake_emails(unidecode(str(vor)), unidecode(str(nach)))

    # Ausgabe der generierten E-Mail-Adresse
    #print(generated_email)


In [ ]:
df = pd.read_excel("Testcases.xlsx")

#replace Umlaute
df["Vorname"] = df['Vorname'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('Ä', 'Ae').str.replace('Ö', 'Oe').str.replace('Ü', 'Ue')
df["Nachname"] = df['Nachname'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('Ä', 'Ae').str.replace('Ö', 'Oe').str.replace('Ü', 'Ue')

def simple(full_name):
    namen = {}

    for index, name in full_name.items():
        if name == "":
            continue

        if name in namen:
            namen[name].append(int(df.at[index, "ID"]))
        else:
            namen[name] = [int(df.at[index, "ID"])]

    df["Simple Match"] = str(0)
    for n in namen:
        for m in namen[n]:
            df.at[m-1, "Simple Match"] = str(namen[n])[1:-1]


    sorted_dict = dict(sorted(namen.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import unidecode
import pandas as pd

def soundex(full_name):
    soundex_mapping = {
        "a": "0", "e": "0", "i": "0", "o": "0", "u": "0", "ä": "0", "ö": "0", "ü": "0", "y": "0", "j": "0", "h": "0",
        "b": "1", "p": "1", "f": "1", "v": "1", "w": "1",
        "c": "2", "s": "2", "k": "2", "g": "2", "q": "2", "x": "2", "z": "2", "ß": "2",
        "d": "3", "t": "3",
        "l": "4",
        "m": "5", "n": "5",
        "r": "6",
        "ch": "7", "sch": "7", "sh": "7",
        "-": " "
    }

    soundex_codes = {}

    for index, name in full_name.items():
        if name == "":
            continue

        name = name.lower()
        name = name.replace("sch", "7").replace("ch", "7").replace("sh", "7")

        soundex_code = name[0].upper()

        previous_digit = ""
        for letter in name[1:]:
            if letter in soundex_mapping:
                current_digit = soundex_mapping[letter]
                if current_digit != previous_digit and current_digit != "0":
                    soundex_code += current_digit
                    previous_digit = current_digit

        soundex_code = soundex_code[:4]

        while len(soundex_code) < 4:
            soundex_code += '0' 
        if soundex_code in soundex_codes:
            soundex_codes[soundex_code].append(index+1)
        else:
            soundex_codes[soundex_code] = [index+1]
            
    df["Soundex"] = str(0)
    
    for n in soundex_codes:
        for m in soundex_codes[n]:
            df.at[m-1, "Soundex"] = str(soundex_codes[n])[1:-1]



In [ ]:
#Soundex representation
df = pd.read_excel("Testcases.xlsx")

def soundex_custom(full_name):
    soundex_mapping = {
            "a": "0", "e": "0", "i": "0", "o": "0", "u": "0", "ä": "0", "ö": "0", "ü": "0", "y": "0", "j": "0", "h": "0",
            "b": "1", "p": "1", "f": "1", "v": "1", "w": "1",
            "c": "2", "s": "2", "k": "2", "g": "2", "q": "2", "x": "2", "z": "2", "ß": "2",
            "d": "3", "t": "3",
            "l": "4",
            "m": "5", "n": "5",
            "r": "6",
            "ch": "7", "sch": "7",
            "-": " "
        }

    soundex_namen = {}

    for index, name in full_name.items():

        if name == "" or pd.isna(name):
            continue

        name = str(name).replace("sch", "7").replace("ch", "7").replace("sh", "7")

        soundex_name = ""
        soundex_name += name[0]

        for letter in name.lower()[1:]:
            if letter in soundex_mapping and soundex_mapping[letter] != soundex_name[-1]:
                soundex_name += soundex_mapping[letter]

            elif letter not in soundex_mapping:
                soundex_name += letter
                
        if soundex_name in soundex_namen:
            soundex_namen[soundex_name].append(int(df.at[index, "ID"]))
        else:
            soundex_namen[soundex_name] = [int(df.at[index, "ID"])]

    df["Soundex Custom"] = str(0)
    for n in soundex_namen:
        for m in soundex_namen[n]:
            df.at[m-1, "Soundex Custom"] = str(soundex_namen[n])[1:-1]
        

In [ ]:
def levenshteinDistance(A, B):
    N, M = len(A), len(B)
    
    dp = [[0 for i in range(M + 1)] for j in range(N + 1)]

    for j in range(M + 1):
        dp[0][j] = j
        
    for i in range(N + 1):
        dp[i][0] = i
        
    for i in range(1, N + 1):
        for j in range(1, M + 1):
            if A[i - 1] == B[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(
                    dp[i-1][j],
                    dp[i][j-1],
                    dp[i-1][j-1]
                )

    return dp[N][M]

In [ ]:
#replace Umlaute

df["Vorname"] = df['Vorname'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('Ä', 'Ae').str.replace('Ö', 'Oe').str.replace('Ü', 'Ue')
df["Nachname"] = df['Nachname'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('Ä', 'Ae').str.replace('Ö', 'Oe').str.replace('Ü', 'Ue')


def levenshtein(full_name, max_distance=3):
    namen = {}
    max_distance = 3

    for index, name in full_name.items():
        if pd.isna(name):
            continue
        if name not in namen:
            namen[name] = [int(df.at[index, "ID"])]
        elif pd.notna(df.at[index, "ID"]):
            namen[name].append(int(df.at[index, "ID"]))

    for name in namen:
        for n in namen:
            if any(namen[n]) not in namen[name]:
                if levenshteinDistance(name, n) in range(1, max_distance):
                    namen[name] += namen[n]
                    namen[name] = list(set(namen[name]))


    df["Levenshtein"] = str(0)
    for n in namen:
        for m in namen[n]:
            df.at[m-1, "Levenshtein"] = str(namen[n])[1:-1]


In [ ]:
def koln(df, column_name):
    koln_mapping = {
        "a": "", "e": "", "i": "", "o": "", "u": "", "ä": "", "ö": "", "ü": "", "y": "", "j": "", "h": "",
        "b": "1",
        "f": "3", "v": "3", "w": "3",
        "g": "4", "k": "4", "q": "4",
        "l": "5",
        "m": "6", "n": "6",
        "r": "7",
        "s": "8", "z": "8",
        "-": ""
    }

    def koln_encode(name):
        if name != "":

            koln_name = [str(name)[0]]

            for n, letter in enumerate(str(name)[1:], start=1):
                if letter == "p":
                    if n < len(name) and name[n] == "h":
                        koln_name.append("3")
                    else:
                        koln_name.append("1")

                elif letter in "dt":
                    if n < len(name) and name[n] in "csz":
                        koln_name.append("8")
                    else:
                        koln_name.append("2")

                elif letter == "c":
                    if n == 1 and n < len(name) and name[n] in "ahkloqrux":
                        koln_name.append("4")
                    elif n < len(name) and name[n] in "ahkloqrux" and name[n - 1] not in "sz":
                        koln_name.append("4")
                    else:
                        koln_name.append("8")

                elif letter == "x":
                    if name[n - 1] in "ckq":
                        koln_name.append("8")
                    else:
                        koln_name.append("48")

                elif letter in koln_mapping and (not koln_name or koln_mapping[letter] != koln_name[-1]):
                    koln_name.append(koln_mapping[letter])

            koln_name = ''.join(koln_name[i] for i in range(len(koln_name)) if i == 0 or koln_name[i] != koln_name[i-1])
            return koln_name
        

    df["Koln"] = df[column_name].apply(koln_encode)

    koln_namen = df.groupby("Koln")["ID"].apply(list).to_dict()

    df["Koln"] = df["Koln"].map(lambda k: ", ".join(map(str, koln_namen.get(k, []))) or "0")


In [ ]:
def ngrams(text, n=2):
    text = f"{'#' * (n-1)}{text.lower().replace(' ', '_')}{'#' * (n-1)}"
    return [text[i:i+n] for i in range(len(text) - n + 1)]

def cosine_similarity(ngrams1, ngrams2):
    freq_vector1 = Counter(ngrams1)
    freq_vector2 = Counter(ngrams2)

    unique_ngrams = set(freq_vector1.keys()) | set(freq_vector2.keys())

    dot_product = sum(freq_vector1[ngram] * freq_vector2[ngram] for ngram in unique_ngrams)

    magnitude1 = np.sqrt(sum(freq_vector1[ngram] ** 2 for ngram in unique_ngrams))
    magnitude2 = np.sqrt(sum(freq_vector2[ngram] ** 2 for ngram in unique_ngrams))

    similarity = dot_product / (magnitude1 * magnitude2)

    return similarity


def ngram_similarity(full_name, ngram=3, border=0.65):
    df["Ngram Similarity"] = str(0)
    similarities = {}
    names = full_name.to_list()

    for i in range(len(names)):
        for j in range(i + 1, len(names)):
            ngrams1 = ngrams(names[i], ngram)
            ngrams2 = ngrams(names[j], ngram)
            if cosine_similarity(ngrams1, ngrams2) > border:
                if i not in similarities:
                    similarities[i] = [i+1, j+1]
                else:
                    similarities[i].append(j+1)
                
                print(cosine_similarity(ngrams1, ngrams2), names[i], names[j])

        if i not in similarities:
            similarities[i] = [i+1]

    for i in similarities:
        for j in similarities[i]:
            if df.at[j-1, "Ngram Similarity"] == str(0):
                df.at[j-1, "Ngram Similarity"] = str(similarities[i])[1:-1]

In [ ]:
def minhash_for_ngrams(ngrams, num_perm=128):
    mhash = MinHash(num_perm=num_perm)
    for ngram in ngrams:
        mhash.update(ngram.encode("utf-8"))
    return mhash

def lsh_similarity_check(full_name, ngram_size=2, threshold=0.65, num_perm=128):
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    minhashes = {}
    similarities = []

    for index, name in full_name.items():
        if name == "":
            continue
        name_ngrams = ngrams(str(name), ngram_size)
        mhash = minhash_for_ngrams(name_ngrams, num_perm)
        lsh.insert(f"name{index}", mhash)
        minhashes[f"name{index}"] = mhash

    df["LSH Similarity"] = pd.Series([set() for _ in range(len(df))])

    for key, minhash in minhashes.items():
        potential_matches = lsh.query(minhash)
        index_key = int(key.replace("name", ""))
        for match in potential_matches:
            if key != match:
                index_match = int(match.replace("name", ""))
                actual_similarity = minhash.jaccard(minhashes[match])
                if actual_similarity >= threshold:
                    similarities.append(actual_similarity)
                    df.at[index_key, "LSH Similarity"].add(index_match+1)
                    df.at[index_match, "LSH Similarity"].add(index_key+1)
            elif key == match:
                df.at[index_key, "LSH Similarity"].add(index_key+1)
                
                
    df["LSH Similarity"] = df["LSH Similarity"].apply(lambda x: str(sorted(list(x)))[1:-1] or "0")


def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    if len(union) == 0:
        return 1.0
    jaccard_index = len(intersection) / len(union)
    
    return jaccard_index

print(jaccard_similarity("Henri Waternoose", "Henry Peter Waternose"))
print(cosine_similarity(ngrams("johann johann", 3), ngrams("johann seeler", 3)))


print(jaccard_similarity(ngrams("robert frei", 2), ngrams("robert frei robert", 2)))
print(jaccard_similarity(ngrams("Henri Waternoose", 2), ngrams("Henry Peter Waternose", 2)))
print(cosine_similarity(ngrams("Henri Waternoose", 3), ngrams("Henry Peter Waternose", 3)))

print(jaccard_similarity(ngrams("mario mueller", 3), ngrams("maria mueller", 3)))
print(jaccard_similarity(ngrams("juergen kaiser", 3), ngrams("juergen geiser", 3)))

In [ ]:
"""
Rerun all algorithms, if "TypeError: 'tuple' object is not callable" appears

"""

df = pd.read_excel("Testcases.xlsx")
import time

vornamen = unidecoded(df["Vorname"]).to_list()
nachnamen = unidecoded(df["Nachname"]).to_list()
emails = unidecoded(df["Email"]).to_list()


df["Standardized"] = str(0)
index = 0
for vor, nach, email in zip(vornamen, nachnamen, emails):
    full_name = replace_non_letters(f"{vor} {nach}".lower())
    email_prefix = email.split("@")[0].lower()
    email_prefix = replace_non_letters(re.sub(r'\d+', '', email_prefix))
    
    if vor == "nan" or nach == "nan":
        if email != "nan":
            to_check = email.split(".")[0]
                                                               
        else:
            to_check = ""
            
    elif plausible_email(vor, nach, email):
        to_check = full_name
    
    elif cosine_similarity(ngrams(full_name, 2), ngrams(email_prefix, 2)) > 0.65:
        to_check = email_prefix
        print(to_check, "|", full_name)
    
    else:
        to_check = full_name
    df.at[index, "Standardized"] = to_check
    index += 1
    
simple(df["Standardized"])
soundex(df["Standardized"])
soundex_custom(df["Standardized"])
koln(df, "Standardized")
levenshtein(df["Standardized"])
ngram_similarity(df["Standardized"], 3)
start = time.time()
lsh_similarity_check(df["Standardized"], 2)
end = time.time()

print(end - start)
df.to_excel("output.xlsx")

In [ ]:
df = pd.read_excel("output.xlsx")

simple = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Simple Match"].to_list())
soundex = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex"].to_list())
soundex_custom = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex Custom"].to_list())
koln = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Koln"].to_list())
ls = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Levenshtein"].to_list())
cos = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Ngram Similarity"].to_list())
lsh = measure_metrics2(df["Duplikat mit Id"].to_list(), df["LSH Similarity"].to_list())

simple, soundex, soundex_custom, koln, ls, cos, lsh

In [ ]:
"""
Rerun all algorithms, if "TypeError: 'tuple' object is not callable" appears

"""


df = pd.read_csv("restaurants.tsv", sep="\t")
import time
print(df)
namen = unidecoded(df["name"]).to_list()

df["Standardized"] = str(0)
index = 0
for name in namen:
    df.at[index, "Standardized"] = replace_non_letters(name)

    index += 1
    
simple(df["Standardized"])
soundex(df["Standardized"])
soundex_custom(df["Standardized"])
koln(df, "Standardized")
levenshtein(df["Standardized"])
ngram_similarity(df["Standardized"])
start = time.time()
lsh_similarity_check(df["Standardized"])
end = time.time()

print(end - start)
df.to_excel("output - restaurant.xlsx")

In [ ]:
df = pd.read_excel("output - restaurant.xlsx")

simple = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Simple Match"].to_list(), include_non_dups=True)
soundex = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex"].to_list(), include_non_dups=True)
soundex_custom = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex Custom"].to_list(), include_non_dups=True)
koln = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Koln"].to_list(), include_non_dups=True)
ls = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Levenshtein"].to_list(), include_non_dups=True)
cos = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Ngram Similarity"].to_list(), include_non_dups=True)
ngram = measure_metrics2(df["Duplikat mit Id"].to_list(), df["LSH Similarity"].to_list(), include_non_dups=True)

simple, soundex, soundex_custom, koln, ls, cos, ngram

In [ ]:
"""
Rerun all algorithms, if "TypeError: 'tuple' object is not callable" appears

"""

df = pd.read_csv("cora.tsv", sep="\t")
import time

authors = unidecoded(df["authors"]).to_list()
title = unidecoded(df["authors"]).to_list()

df["Standardized"] = str(0)
index = 0
for author, tit in zip(authors, title):
    
    if author == "nan":
        to_check = tit
    
    else:
        to_check = author
    df.at[index, "Standardized"] = to_check
    index += 1
    
simple(df["Standardized"])
soundex(df["Standardized"])
soundex_custom(df["Standardized"])
koln(df, "Standardized")
levenshtein(df["Standardized"])
ngram_similarity(df["Standardized"])
start = time.time()
lsh_similarity_check(df["Standardized"])
end = time.time()

print(end - start)
df.to_excel("output - cora.xlsx")

In [ ]:
df = pd.read_excel("output - cora.xlsx")

simple = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Simple Match"].to_list())
soundex = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex"].to_list())
soundex_custom = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Soundex Custom"].to_list())
koln = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Koln"].to_list())
ls = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Levenshtein"].to_list())
cos = measure_metrics2(df["Duplikat mit Id"].to_list(), df["Ngram Similarity"].to_list())
ngram = measure_metrics2(df["Duplikat mit Id"].to_list(), df["LSH Similarity"].to_list())

simple, soundex, soundex_custom, koln, ls, cos, ngram